<a href="https://colab.research.google.com/github/Aryan-B25/CSC637/blob/main/Yelp_Text_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.listdir("/content")


['.config', 'drive', 'sample_data']

In [ ]:
file_path = "/content/drive/MyDrive/Assignment/Yelp JSON/yelp_academic_dataset_review.json"
import json

with open(file_path, "r", encoding="utf-8") as f:
    for _ in range(5):
        print(json.loads(f.readline()))


{'review_id': 'KU_O5udG6zpxOg-VcAEodg', 'user_id': 'mh_-eMZ6K5RLWhZyISBhwA', 'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw', 'stars': 3.0, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", 'date': '2018-07-07 22:09:11'}
{'review_id': 'BiTunyQ73aT9WBnpR9DZGw', 'user_id': 'OyoGAe7OKpv6SyGZT5g77Q', 'business_id': '7ATYjTIgM3jUlt4UM3IypQ', 'stars': 5.0, 'useful': 1, 'funny': 0, 'cool': 1, 'text': "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycl

In [ ]:
file_path = "/content/drive/MyDrive/Assignment/Yelp JSON/yelp_academic_dataset_review.json"

# Counting total number of lines (reviews) in the file
total_reviews = sum(1 for _ in open(file_path, "r", encoding="utf-8"))

print(f"Total number of reviews in the dataset: {total_reviews}")


Total number of reviews in the dataset: 6990280


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
nltk.download('all')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_en

True

In [ ]:
import json
import re
import nltk
import multiprocessing
import time
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob


In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


In [ ]:
def clean_text(review):
    text = review.get("text", "").lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove non-alphabetic characters
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    review["cleaned_text"] = " ".join(tokens)  # Store cleaned text
    return review


In [ ]:
file_path = "/content/drive/MyDrive/Assignment/Yelp JSON/yelp_academic_dataset_review.json"
num_reviews = 1000000

with open(file_path, "r", encoding="utf-8") as f:
    reviews = [json.loads(f.readline()) for _ in range(num_reviews)]


In [ ]:
start_time = time.time()
single_threaded_reviews = [clean_text(review) for review in reviews]
single_time = time.time() - start_time
print(f"Single-threaded processing time: {single_time:.2f} sec")


Single-threaded processing time: 698.47 sec


In [ ]:
start_time = time.time()
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
parallel_reviews = pool.map(clean_text, reviews)
pool.close()
pool.join()
parallel_time = time.time() - start_time
print(f"Parallel processing time: {parallel_time:.2f} sec")


Parallel processing time: 668.39 sec


In [ ]:

print(f"Single-threaded time: {single_time:.2f} sec")
print(f"Parallel processing time: {parallel_time:.2f} sec")



Single-threaded time: 698.47 sec
Parallel processing time: 668.39 sec


In [ ]:
df = pd.DataFrame(parallel_reviews)
df.to_csv("/content/cleaned_yelp_reviews_1M.csv", index=False)
print("Processed 1,000,000 reviews saved as CSV!")


Processed 1,000,000 reviews saved as CSV!


In [ ]:
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

df["sentiment"] = df["cleaned_text"].apply(lambda x: get_sentiment(str(x)))
df[["cleaned_text", "sentiment"]].head()


,cleaned_text,sentiment
0,decide eat aware going take hour beginning end...,0.198485
1,ive taken lot spin class year nothing compare ...,0.395455
2,family diner buffet eclectic assortment large ...,0.139935
3,wow yummy different delicious favorite lamb cu...,0.315152
4,cute interior owner gave u tour upcoming patio...,0.416234
